In [1]:
from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import concatenate
from keras.layers import GRU
from keras.layers import Dense
from keras.models import Model
from keras.layers import MaxPooling1D
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.utils import shuffle
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np
import pickle
import scipy.io as sio
from scipy.signal import resample
import numpy as np
from numpy.random import choice
def window_eeg(x, window_sec, stride_sec, data_len_sec, samp_freq):
    nchannel = x.shape[0]
    npoint_original = x.shape[1]
    nwindow = int((data_len_sec - window_sec)/stride_sec + 1)
    #print(nwindow)
    window_pt = int(samp_freq*window_sec)
    windowed_eeg = np.zeros((nwindow,window_pt,nchannel))
    for i in range(nwindow):
        x_windowed = x[:,int(np.ceil(i*stride_sec*samp_freq)):int(np.ceil((i*stride_sec+window_sec)*samp_freq))]
        #print(x_windowed.shape)
        x_windowed = np.transpose(x_windowed)
        windowed_eeg[i,:,:] = x_windowed
    return windowed_eeg
def down_sample(x, samp_freq, target_freq):
    nchannel = x.shape[0]
    original_len = x.shape[1]
    target_len = int(original_len*target_freq/samp_freq)
    target_x = np.zeros((nchannel,target_len))
    for i in range(nchannel):
        target_x[i,:] = resample(x[i,:], target_len)
    return target_x
window = 60
stride = window
target_freq = 200
data_len_sec = 600
nwindow = int((data_len_sec - window)/stride + 1)

samp_freq = 200
window = 60
nchannel = 16
data_len = int(samp_freq*window)
print(data_len)
#data_len = 15000
input_eeg_raw = Input(shape=(data_len, nchannel))

Using TensorFlow backend.


12000


In [2]:
def AES_test_train_split_left1out(subject, total_preictal, total_interictal, train_preictal, nth):
    window = 60
    stride = window
    target_freq = 200
    data_len_sec = 600
    nwindow = int((data_len_sec - window)/stride + 1)
    
    npreictal = total_preictal
    ninterictal = total_interictal
    preictal_idx = [i for i in range(npreictal)]
    interictal_idx = [i for i in range(ninterictal)]
    n_preictal_train = train_preictal
    n_interictal_train = int(n_preictal_train*2)
    train_preictal_idx = [i for i in range(npreictal) if i != nth]
    train_preictal_mat_idx = []
    for i in train_preictal_idx:
        for j in np.arange(i*6,i*6+6):
            train_preictal_mat_idx.append(j)
    train_interictal_idx = choice(interictal_idx, n_interictal_train, replace = False)
    train_interictal_mat_idx = []
    for i in train_interictal_idx:
        for j in np.arange(i*6,i*6+6):
            train_interictal_mat_idx.append(j)
    test_preictal_idx = [nth]
    test_interictal_idx = np.setdiff1d(interictal_idx, train_interictal_idx)
    test_interictal_idx = choice(test_interictal_idx, int((npreictal-n_preictal_train)*2), replace = False)
    test_preictal_mat_idx = []
    for i in test_preictal_idx:
        for j in np.arange(i*6,i*6+6):
            test_preictal_mat_idx.append(j)
    test_interictal_mat_idx = []
    for i in test_interictal_idx:
        for j in np.arange(i*6,i*6+6):
            test_interictal_mat_idx.append(j)   
    X_train = []
    Y_train = []
    for i in train_interictal_mat_idx:
        filename = '{0}\\{1}_interictal_segment_{2}.mat'.format(subject, subject, str(i+1).zfill(4))
        file = sio.loadmat(filename)
        key = 'interictal_segment_{0}'.format(str(i+1)) 
        x = file[key]['data'][0,0]
        data_len_sec = file[key]['data_length_sec'][0,0][0,0]
        samp_freq = file[key]['sampling_frequency'][0,0][0,0]
        x_down = down_sample(x, samp_freq, target_freq)
        x_windowed = window_eeg(x_down, window, stride, data_len_sec, target_freq)
        for j in range(x_windowed.shape[0]):
            X_train.append(x_windowed[j,:,:])
            Y_train.append(0)
    for i in train_preictal_mat_idx:
        filename = '{0}\\{1}_preictal_segment_{2}.mat'.format(subject, subject, str(i+1).zfill(4))
        file = sio.loadmat(filename)
        key = 'preictal_segment_{0}'.format(str(i+1)) 
        x = file[key]['data'][0,0]
        data_len_sec = file[key]['data_length_sec'][0,0][0,0]
        samp_freq = file[key]['sampling_frequency'][0,0][0,0]
        x_down = down_sample(x, samp_freq, target_freq)
        x_windowed = window_eeg(x_down, window, stride, data_len_sec, target_freq)
        for j in range(x_windowed.shape[0]):
            X_train.append(x_windowed[j,:,:])
            Y_train.append(1)
    X_test = []
    Y_test = []
    for i in test_interictal_mat_idx:
        filename = '{0}\\{1}_interictal_segment_{2}.mat'.format(subject, subject, str(i+1).zfill(4))
        file = sio.loadmat(filename)
        key = 'interictal_segment_{0}'.format(str(i+1)) 
        x = file[key]['data'][0,0]
        data_len_sec = file[key]['data_length_sec'][0,0][0,0]
        samp_freq = file[key]['sampling_frequency'][0,0][0,0]
        x_down = down_sample(x, samp_freq, target_freq)
        x_windowed = window_eeg(x_down, window, stride, data_len_sec, target_freq)
        for j in range(x_windowed.shape[0]):
            X_test.append(x_windowed[j,:,:])
            Y_test.append(0)
    for i in test_preictal_mat_idx:
        filename = '{0}\\{1}_preictal_segment_{2}.mat'.format(subject, subject, str(i+1).zfill(4))
        file = sio.loadmat(filename)
        key = 'preictal_segment_{0}'.format(str(i+1)) 
        x = file[key]['data'][0,0]
        data_len_sec = file[key]['data_length_sec'][0,0][0,0]
        samp_freq = file[key]['sampling_frequency'][0,0][0,0]
        x_down = down_sample(x, samp_freq, target_freq)
        x_windowed = window_eeg(x_down, window, stride, data_len_sec, target_freq)
        for j in range(x_windowed.shape[0]):
            X_test.append(x_windowed[j,:,:])
            Y_test.append(1)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    print(train_preictal_idx, train_interictal_idx, test_preictal_idx, test_interictal_idx)
    processed_data = {'X_train':X_train, 'Y_train':Y_train, 'X_test': X_test, 'Y_test': Y_test}
    return processed_data

In [6]:
def start_chrononet():
    act_func = 'relu'
    input_eeg = Dropout(0)(input_eeg_raw)
    conv11 = Conv1D(filters = 32, strides = 2, kernel_size = 2, padding = 'same', activation = act_func)(input_eeg)
    conv12 = Conv1D(filters = 32, strides = 2, kernel_size = 4, padding = 'same', activation = act_func)(input_eeg)
    conv13 = Conv1D(filters = 32, strides = 2, kernel_size = 8, padding = 'same', activation = act_func)(input_eeg)
    #print(conv11.shape,conv12.shape,conv13.shape)
    conv1_concate = concatenate([conv11, conv12, conv13], axis = 2)
    conv1_concate = Dropout(0)(conv1_concate)
    #print(conv1_concate.shape)
    conv21 = Conv1D(filters = 32, strides = 2, kernel_size = 2, padding = 'same', activation = act_func)(conv1_concate)
    conv22 = Conv1D(filters = 32, strides = 2, kernel_size = 4, padding = 'same', activation = act_func)(conv1_concate)
    conv23 = Conv1D(filters = 32, strides = 2, kernel_size = 8, padding = 'same', activation = act_func)(conv1_concate)
    conv2_concate = concatenate([conv21, conv22, conv23], axis = 2)
    conv2_concate = Dropout(0)(conv2_concate)
    print(conv2_concate.shape)
    conv31 = Conv1D(filters = 32, strides = 2, kernel_size = 2, padding = 'same', activation = act_func)(conv2_concate)
    conv32 = Conv1D(filters = 32, strides = 2, kernel_size = 4, padding = 'same', activation = act_func)(conv2_concate)
    conv33 = Conv1D(filters = 32, strides = 2, kernel_size = 8, padding = 'same', activation = act_func)(conv2_concate)
    conv3_concate = concatenate([conv31, conv32, conv33], axis = 2)
    conv3_concate = Dropout(0)(conv3_concate)
    print(conv3_concate.shape)
    gru1 = GRU(units = 32, return_sequences = True)(conv3_concate)
    gru2 = GRU(units = 32, return_sequences = True)(gru1)
    gru_concate_1 = concatenate([gru1, gru2], axis = 1)
    gru3 = GRU(units = 32, return_sequences = True)(gru_concate_1)
    gru_concate_2 = concatenate([gru1, gru2, gru3], axis = 1)
    gru_4 = GRU(units = 32)(gru_concate_2)
    output = Dense(2, activation = 'softmax')(gru_4)
    chrononet_model = Model(inputs = input_eeg_raw, outputs = output)
    chrononet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(chrononet_model.summary)
    return chrononet_model

In [10]:
for left in range(12):
    data = AES_test_train_split_left1out('Dog_3', 12, 240, 11, left)
    print('finished preprocessing data lefting out: ', left)
    seed = 7
    np.random.seed(seed)
    X_train = data['X_train']
    Y_train = data['Y_train']
    X_test = data['X_test']
    Y_test = data['Y_test']
#     X_train, Y_train = shuffle(X_train, Y_train, random_state=0)
#     encoder = LabelEncoder()
#     encoder.fit(Y_train)
#     encoded_Y_train = encoder.transform(Y_train)
#     # convert integers to dummy variables (i.e. one hot encoded)
#     dummy_y_train = np_utils.to_categorical(encoded_Y_train)
#     print(X_train.shape, dummy_y_train.shape)
#     X_test, Y_test = shuffle(X_test, Y_test, random_state=0)
#     encoded_Y_test = encoder.transform(Y_test)
#     # convert integers to dummy variables (i.e. one hot encoded)
#     dummy_y_test = np_utils.to_categorical(encoded_Y_test)
#     chrononet = start_chrononet()
#     history = chrononet.fit(X_train, dummy_y_train, validation_data=(X_test, dummy_y_test), epochs=1, batch_size=128, verbose = 1)
#     chrononet.save('chrononet_dog_3_60epoch_batch64_{0}.h5'.format(left))
#     np.save('chrononet_dog_3_60epoch_batch64_{0}_train_acc'.format(left),history.history['acc'])
#     np.save('chrononet_dog_3_60epoch_batch64_{0}_val_acc'.format(left),history.history['val_acc'])
#     print(left)

finished preprocessing data lefting out:  0
(1980, 12000, 16) (1980, 2)
(?, 3000, 96)
(?, 1500, 96)
<bound method Container.summary of <keras.engine.training.Model object at 0x0000022AEF6A0E10>>
Train on 1980 samples, validate on 180 samples
Epoch 1/1
1980/1980 [==============================] - 263s 133ms/step - loss: 0.6552 - acc: 0.6394 - val_loss: 0.6375 - val_acc: 0.6667


NameError: name 'chrononet_model' is not defined

In [4]:
a = AES_test_train_split_left1out('Dog_3', 12, 240, 11, 11)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] [  4 125  89  75  18  44 112 120  35 103 129 109 197  26 199  97  46   1
 180  53 223 107] [11] [222  76]


In [5]:
test_interictal_mat_idx

NameError: name 'test_interictal_mat_idx' is not defined